In [93]:
import os
import psycopg2 #PosgreSQL adapter for Python
from dotenv import load_dotenv 
import pandas as pd
from io import StringIO

**Create a dataframe**

In [94]:
#create a dataframe
data = {
    'name': ['Martin', 'Alex', 'Lucy', 'Peter', 'David', 'Jane'],
    'surname': ['Bilek', 'Turner', 'White', 'Black', 'Baker', 'Field'],
    'gender': ['male', 'male', 'female', 'male', 'male', 'female'],
    'nationality':['CZE','US', 'UK', 'US', 'UK', 'US'],
    'age': [39, 28, 36, 50, 66, 45]
}

df = pd.DataFrame(data)
df

,name,surname,gender,nationality,age
0,Martin,Bilek,male,CZE,39
1,Alex,Turner,male,US,28
2,Lucy,White,female,UK,36
3,Peter,Black,male,US,50
4,David,Baker,male,UK,66
5,Jane,Field,female,US,45


**Create a connection with the database and open the cursor**

In [95]:
#loads .env file = connects with the database
load_dotenv()

#set up connection with the database (psycopg2 is faster than Alchemy)
url=os.getenv("DATABASE_URL")
connection=psycopg2.connect(url)


#Open a cursor to perform database operations
cursor = connection.cursor()

**Define table**

In [96]:
#define the table
table_name='people'

**Delete previous table**

In [97]:
""" #SQL query to drop the table
DROP_TABLE_QUERY = f"DROP TABLE IF EXISTS {table_name}"


#Excute the operation
cursor.execute(DROP_TABLE_QUERY)

#Commit changes to the database
connection.commit() """

' #SQL query to drop the table\nDROP_TABLE_QUERY = f"DROP TABLE IF EXISTS {table_name}"\n\n\n#Excute the operation\ncursor.execute(DROP_TABLE_QUERY)\n\n#Commit changes to the database\nconnection.commit() '

**Create a table**

In [98]:
#Create a table
CREATE_FIRST_TABLE=f"""CREATE TABLE IF NOT EXISTS {table_name}(
            id SERIAL PRIMARY KEY,
            name VARCHAR(50),
            surname VARCHAR(50),
            gender VARCHAR(50),
            nationality VARCHAR(50),
            age DECIMAL
            )"""

#Excute the operation
cursor.execute(CREATE_FIRST_TABLE)

#Commit changes to the database
connection.commit()

**Check the columns**

In [99]:
#SQL query to read data from the 'people' table
READ_DATA = f"SELECT * FROM {table_name}"

#Execute the query
cursor.execute(READ_DATA)

#Fetch all rows at once
rows = cursor.fetchall()

#Get the column names
columns = [desc[0] for desc in cursor.description]
columns

['id', 'name', 'surname', 'gender', 'nationality', 'age']

**Delete previous data from the table**

In [100]:
#delete previous data from the database

DELETE_DATA = f"DELETE FROM {table_name}"

#Execute the DELETE query
cursor.execute(DELETE_DATA)

#commit
connection.commit()

**Insert data into the table**

In [101]:
#Insert a data into a table
# Create a CSV string from the DataFrame (excluding the id column)
csv_data = df[['name', 'surname', 'gender', 'nationality', 'age']].to_csv(index=False, header=False)

# Use StringIO to create a file-like object from the CSV string
csv_file = StringIO(csv_data)


try:
    # Use the COPY command to load data from the CSV file into the table
    cursor.copy_from(csv_file, table_name, sep=',', columns=('name', 'surname', 'gender', 'nationality', 'age')) #NO capital letters!!!!

except Exception as e:
    print(f"Error: {e}")

#commit
connection.commit()

**Read and filter the data from the table and insert them into a dataframe**

In [102]:
#Read data from the table
READ_DATA=f"SELECT* FROM  {table_name} WHERE gender = 'male' AND  nationality = 'US' "

#Execute the query
cursor.execute(READ_DATA)

# Fetch all rows at once
rows = cursor.fetchall()

#Insert data into a dataframe
columns = [desc[0] for desc in cursor.description] #get names of the columns
df = pd.DataFrame(rows, columns=columns)
df

,id,name,surname,gender,nationality,age
0,8,Alex,Turner,male,US,28
1,10,Peter,Black,male,US,50


**Close the connection with the database**

In [103]:
# Close the cursor and connection 
if cursor:
    cursor.close()

if connection:
    connection.close()